In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load dataset (update the path to your file)
file_path = r"C:\Users\alrec\Desktop\DAT Capstone\combined_nfl_play_by_play_2003_2023.csv"
df = pd.read_csv(file_path)

C:\Users\alrec\AppData\Local\Temp\ipykernel_47364\1573016920.py:3: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [5]:
# Display first few rows of the dataframe
print("\nFirst 5 rows:")
print(df.head())

# Display number of rows and columns
num_rows, num_columns = df.shape
print(f"Number of Rows: {num_rows}")
print(f"Number of Columns: {num_columns}")
columns = pd.DataFrame(df.columns, columns=["Column Name"])


# Check for missing values
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_df = pd.DataFrame({'Column': missing_values.index, 'NA_Count': missing_values.values})
output_path = r"C:/Users/alrec/Desktop/DAT Capstone/nan_count.csv"

# Save missing value count to nan_count.csv
missing_df.to_csv(output_path, index=False)
print("\nWrote number of missing values per column to nan_count.csv")


# Check for duplicate rows
print("\nNumber of duplicate rows:", df.duplicated().sum())


First 5 rows:
   play_id          game_id home_team away_team season_type  week posteam  \
0        1  2003_01_ARI_DET       DET       ARI         REG     1     NaN   
1       35  2003_01_ARI_DET       DET       ARI         REG     1     DET   
2       57  2003_01_ARI_DET       DET       ARI         REG     1     DET   
3       78  2003_01_ARI_DET       DET       ARI         REG     1     DET   
4       99  2003_01_ARI_DET       DET       ARI         REG     1     DET   

  posteam_type side_of_field  yardline_100  ... total_line    surface  temp  \
0          NaN           NaN           NaN  ...       39.0  fieldturf   NaN   
1         home           ARI          30.0  ...       39.0  fieldturf   NaN   
2         home           DET          70.0  ...       39.0  fieldturf   NaN   
3         home           DET          61.0  ...       39.0  fieldturf   NaN   
4         home           DET          59.0  ...       39.0  fieldturf   NaN   

   wind pass rush  first_down special  play    

In [6]:
# drop st_play_type column
df_trimmed = df.drop(columns=['st_play_type'])

# filter out entries that aren't a play
filter_values = ["GAME", "END QUARTER 1", "END QUARTER 2", "END QUARTER 3", "END GAME"]
df_filtered = df_trimmed[~df_trimmed['desc'].isin(filter_values)]
df_filtered = df_filtered[~df_filtered['desc'].str.contains(r'Timeout', na=False)]


# Check for missing values
missing_values = df_filtered.isnull().sum().sort_values(ascending=False)
missing_df = pd.DataFrame({'Column': missing_values.index, 'NA_Count': missing_values.values})

# Save missing value count to nan_count_filtered.csv
output_path = r"C:/Users/alrec/Desktop/DAT Capstone/nan_count_filtered.csv"
missing_df.to_csv(output_path, index=False)
print("\nWrote number of missing values per column to nan_count_filtered.csv")


print("Number of rows: ", len(df_filtered))


Wrote number of missing values per column to nan_count_filtered.csv
Number of rows:  926838


In [10]:
# One-hot encoding for the specified columns
df_encoded = pd.get_dummies(df_filtered, columns=['field_goal_result', 'extra_point_result', 'two_point_conv_result'], drop_first=True)

# Output new df to df_encoded.csv
output_path = r"C:/Users/alrec/Desktop/DAT Capstone/df_encoded.csv"
df_encoded.to_csv(output_path, index=False)
print("Wrote df_encoded to csv")


Wrote df_encoded to csv


In [11]:
# aggregate data by game and team
aggregated = df_encoded.groupby(['game_id', 'posteam']).agg({
    'yards_gained': 'sum',
    'receiving_yards': 'sum',
    'rushing_yards': 'sum',
    'return_yards': 'sum',
    'incomplete_pass': 'sum',
    'complete_pass': 'sum',
    'interception': 'sum',
    'pass_attempt': 'sum',
    'rush_attempt': 'sum',
    'touchdown': 'sum',
    'pass_touchdown': 'sum',
    'rush_touchdown': 'sum',
    'return_touchdown': 'sum',
    'first_down': 'sum',
    'play': 'sum',
    'sack': 'sum',
    'fumble': 'sum',
    'fumble_lost': 'sum',
    'field_goal_result_made': 'sum',
    'field_goal_result_missed': 'sum',
    'extra_point_result_good': 'sum',
    'extra_point_result_failed': 'sum',
    'safety': 'sum',
    'total_home_epa': 'max',
    'total_away_epa': 'max',
    'total_home_score': 'max',
    'total_away_score': 'max',
    'total': 'max'
}).reset_index()

# Make a copy of the dataframe but swap the teams to calculate defensive stats
df_defense = df_encoded.copy()

# Swap posteam with the opposing team to get the defensive stats for the right team
df_defense['posteam'] = df_defense.apply(
    lambda row: row['home_team'] if row['posteam'] == row['away_team'] else row['away_team'], axis=1)

# Aggregate the data by game and posteam (now representing the defensive side)
aggregated_defense = df_defense.groupby(['game_id', 'posteam']).agg({
    'yards_gained': 'sum',
    'first_down': 'sum',
    'touchdown': 'sum',
    'play': 'sum',
    'interception': 'sum',
    'fumble_forced': 'sum',
    'fumble_lost': 'sum',
    'total_home_score': 'max',
    'total_away_score': 'max',
}).reset_index()

# Rename columns to make it clear these are defensive stats
aggregated_defense.rename(columns={
    'yards_gained': 'yards_allowed',
    'first_down': 'first_down_allowed',
    'touchdown': 'touchdowns_allowed',
    'interception': 'turnovers_gained',
    'fumble_forced': 'fumbles_forced',
    'fumble_lost': 'fumbles_recovered',
}, inplace=True)

# Now merge this defensive aggregated dataframe back with the original offensive one
aggregated = aggregated.merge(aggregated_defense, on=['game_id', 'posteam'], suffixes=('', '_defense'))

# Add offensive calculated stats: yards per play, points per play, first down rate, and number of turnovers (lost)
aggregated['yards_per_play_offense'] = aggregated['yards_gained'] / aggregated['play']
aggregated['points_per_play_offense'] = (aggregated['total_home_score'] + aggregated['total_away_score']) / aggregated['play']
aggregated['first_down_rate_offense'] = aggregated['first_down'] / aggregated['play']
aggregated['turnovers_lost'] = aggregated['interception'] + aggregated['fumble_lost']

# Add defensive calculated stats: yards per play allowed, points per play allowed, first down rate allowed, and turnovers gained
aggregated['yards_per_play_allowed'] = aggregated['yards_allowed'] / aggregated['play_defense']
aggregated['points_per_play_allowed'] = (aggregated['total_home_score_defense'] + aggregated['total_away_score_defense']) / aggregated['play_defense']
aggregated['first_down_rate_allowed'] = aggregated['first_down_allowed'] / aggregated['play_defense']

# Turnovers gained by defense (interceptions + fumbles recovered)
aggregated['turnovers_gained'] = aggregated['turnovers_gained'] + aggregated['fumbles_recovered']

# add special teams calculated stats: fg percentage, xp percentage
aggregated['fg_percentage'] = aggregated['field_goal_result_made'] / (
        aggregated['field_goal_result_made'] + aggregated['field_goal_result_missed'])
aggregated['xp_percentage'] = aggregated['extra_point_result_good'] / (
        aggregated['extra_point_result_good'] + aggregated['extra_point_result_failed'])

# Add turnover differential
aggregated['turnover_differential'] = aggregated['turnovers_gained'] - aggregated['turnovers_lost']


output_path = r"C:/Users/alrec/Desktop/DAT Capstone/df_transform.csv"
aggregated.to_csv(output_path, index=False)
